Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "양동훈"
COLLABORATORS = "PYTHON"

---

# Exercise: Gauss Eliniation, LU Decomposition
**강좌**: *Numerical Analysis*

**Due**: 2024/10/21

## Problem #1
Write a function to solve $Ax=b$ using Gauss Elimination with Partial Pivoting. The function should follow these specifications:

- Input parameters
   * Matrix *A*
   * vector *b*
   
- Return parameters
   * solution vector *x*
   * Pivot indices *l* (indicating row swaps during pivoting)
   
- Additionally, include a proper docstring to describe the function's behavior, parameters, and return values.

In [17]:
import numpy as np

#square한 matrix에 대해서만 사용가능
def pv_gauss_eliminate(A, b):
    # YOUR CODE HERE
    """
    Gauss Elimination

    parameters
    ---------
    A : matrix
        Linear Operator
    b : array 
        Results

    Retruns
    --------
    x : array
        Solution
    """
    m, n = np.array(A).shape
    lenth = len(b)
    pivot_index = list(range(lenth))
    # if (m != n) or (m == 0) or (n == 0) or (n != lenth) or (m != lenth):
        # raise ValueError("Wrong Shape")
    
    #forward Elimination
    for i in range(lenth-1) :
         max_index = np.argmax(abs(A[pivot_index[i:],i])) + i
         pivot_index[max_index], pivot_index[i] = pivot_index[i], pivot_index[max_index]  
         for k in pivot_index[i+1:] :
             ratio = A[k, i]/A[pivot_index[i],i]
             A[k,:] = A[k,:] - ratio*A[pivot_index[i],:]
             b[k] = b[k] - ratio*b[pivot_index[i]]
    A = A[pivot_index,:]
    b = b[pivot_index]
    #Back subsutition
    x = np.empty_like(b)
    for i in range(lenth-1,-1,-1) :
        x[i] = b[i]
        for j in range(i+1, lenth) :
            x[i] -= A[i,j] * x[j]
        x[i] /= A[i,i]
             
    return x, pivot_index

A = np.array([
    [3, -13, 9, 3],
    [-6, 4, 1, -18],
    [6, -2, 2, 4],
    [12, -8, 6, 10]
])

b = np.array([-19, -34, 16, 26])
pv_gauss_eliminate(A, b)

(array([ 3,  1, -2,  1]), [3, 0, 1, 2])

In [18]:
## Do not remove
A = np.array([
    [3, -13, 9, 3],
    [-6, 4, 1, -18],
    [6, -2, 2, 4],
    [12, -8, 6, 10]
])

b = np.array([-19, -34, 16, 26])
x, l = pv_gauss_eliminate(A, b)

# Check your result
assert np.linalg.norm(x - np.array([3, 1, -2, 1])) < 1e-6
assert np.linalg.norm(l - np.array([3, 0, 1, 2])) < 1e-6

## Problem #2
Write two functions to perform LU decomposition ($A=LU$) and substitutions to solve $Ax=b$ using forward and backward substitutions with the following specifications:

- LU decomposition function
    * Convert matrix A into its LU decomposed form.
    
- LU substitution function
    * Input parameters
       * Matrix *LU* (from the LU decomposition)
       * vector *b* (right-hand side of the equation)
       
    - Return parameters
       * solution vector *x* (final solution to $Ax=b$)
       * Intermediate result of forward substitution $Lc=b$
       
- Additionally, include a proper docstring to describe the function's behavior, parameters, and return values.

In [54]:
def lu_decompose(A):
    # YOUR CODE HERE
    m, n = np.array(A).shape
    for i in range(n) :
        for j in range(i+1, n) :
            ratio = A[j,i]/A[i,i]
            A[j, i+1:] = A[j,i+1:] - ratio*A[i,i+1:]
            A[j, i] = ratio
    return A

def lu_subs(A, b):
    # YOUR CODE HERE
    # Lc = b를 이용해서 c 구하기
    n = b.shape[0]
    c = np.empty_like(b)
    for i in range(n):
        c[i] = b[i]
        for j in range(0,i) :
            c[i] -= A[i,j]*c[j]

    #Ux = c를 이용해서 x구하기
    x = np.empty_like(b)
    for i in range(n-1, -1, -1) :
        x[i] = c[i]
        for j in range(i+1,n) :
            x[i] -= A[i,j]*x[j]
        x[i] /= A[i,i]   

    return x, c

In [55]:
## Do not remove
A = np.array([[2, 1, 1], [4, -6, 0], [-2, 7, 2]])
LU = A.copy()

# LU decomposition
lu_decompose(LU)

# Solve Ax=b
b = np.array([5, -2, 9])
x, c = lu_subs(LU, b)

# Check your result
assert np.linalg.norm(LU - np.array([[2, 1, 1], [2, -8, -2], [-1, -1, 1]])) < 1e-6
assert np.linalg.norm(x - np.array([1, 1, 2])) < 1e-6
assert np.linalg.norm(c - np.array([5, -12, 2])) < 1e-6

## Problem #3
Write a function to compute the inverse of matrix AA using LU decomposition and substitution.

In [67]:
def inverse(A):
    # YOUR CODE HERE
    n = A.shape[0]
    I = np.eye(n)
    cal_i = []
    LU = lu_decompose(A.copy())
    x = np.zeros_like(A,dtype=float)
    c = np.empty_like(A, dtype =float)
    for i in range(n) :
        cal_i.append(I[:,i])

    for i in range(n) :
        x[:, i], c[:, i] = lu_subs(LU, cal_i[i])
    
    return x



A = np.array([[2,1,1],[4,-6,0],[-2,7,2]])
A_inv = inverse(A)
   

print(A_inv@A)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [64]:
## Do not remove
A = np.array([[2,1,1],[4,-6,0],[-2,7,2]])
A_inv = inverse(A)

# Check your result
assert np.linalg.norm(A @ A_inv - np.eye(len(A))) < 1e-6

## Problem #4
Write a function to solve a tri-diagonal system using the Thomas algorithm to solve the following problem:

$$
\begin{bmatrix}
2.01475 & -0.020875 & & \\
-0.020875 & 2.01475 & -0.020875 & \\
& -0.020875 & 2.01475 & -0.020875 \\
& & -0.020875 & 2.01475
\end{bmatrix}
T 
= 
\begin{bmatrix}
4.175 \\ 0 \\ 0 \\ 2.0875
\end{bmatrix}
$$

In [72]:
def thomas(a, b, c, d):
    # YOUR CODE HERE
    n = len(d)
    x = np.empty_like(d)

    for i in range(1, n) :
        w = a[i]/b[i-1]
        b[i] -= w*c[i-1]
        d[i] -= w*d[i-1]

    x[-1] = d[-1]/b[-1]
    for j in range(n-2, -1, -1) :
        x[j] = (d[j]-c[j]*x[j+1])/b[j]

    return x
a = np.array([0, -0.020875, -0.020875, -0.020875])
b = np.array([2.01475, 2.01475, 2.01475, 2.01475])
c = np.array(a[::-1])
d = np.array([4.175, 0, 0, 2.0875])
print(thomas(a,b,c,d))

[2.07244105 0.0215863  0.01096005 1.03622226]


In [ ]:
# Solve the given linear system, store the solution in the vector x.
# print(x)
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Do not remove!!!